In [8]:
import matplotlib.pyplot as plt
%matplotlib inline
import re 
import pandas as pd
import tweepy 
from tweepy import OAuthHandler 
import csv
import string
import nltk
from textblob import TextBlob 
nltk.download('stopwords')
from nltk.corpus import stopwords
n_words= set(stopwords.words('english'))

from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer 
porter = PorterStemmer()

lemmatizer = WordNetLemmatizer() 
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zappe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zappe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\zappe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [130]:
# KELVIN
# CONSUMER_KEY = "HHeGpSt56Grqy6fI6LnImwZLq"
# CONSUMER_SECRET_KEY = "bUmbG22nl4aZSkDqgzK4zotUGvovEoE6pXuP4tyI8uqeivyeAf"

# BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAABaVZQEAAAAArrrymJHIyEf7TkhQZ2MMBUoKfvU%3DI4FCjVMOW6YoUWYxqYEFbpNIB3HiF1796Fq9axACOr4bdYci7T"
# ACCESS_TOKEN = "366489439-e0m7AoT0O9Flc8N8actIan7lN9w0XU0C1uqQ8sXu"
# ACCESS_TOKEN_SECRET = "RBSxBUNHRgZLfpXmzdTN4gu0OVEwnznlCVGmcmguMitky"


# WEI JIE
# CONSUMER_KEY = "SjhDYQQ1eL9MMk3WRanA5fjs1"
# CONSUMER_SECRET_KEY = "Mey39acc7PU6SqwjGI9SdqQMDcmHn8fccGOOIyIrL8g7jjM2dF"

# BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAADCfaAEAAAAAiZBDEALWz5wjmzDdJ8vCJwo5Sbc%3DycI2xIQupis2aISBQNX7dVUiZYFPyhJwcvMtzukeY6Y1vN69I9"
# ACCESS_TOKEN = "569831527-OIUjrplFJ1Sk4n7EarjjEG5RVbLoOsAr0ZCBR7Sw"
# ACCESS_TOKEN_SECRET = "QaSpMAN3AuXrSp6Hqk7dqiyfi7b0pXnyRWHemt7TIvvVg"

CONSUMER_KEY = 'DZO4o46uebmsiM3QijUmbMj1J',
CONSUMER_SECRET_KEY =    'jA9g5FaYIeXwrxTNrBM5zXqVJrjfqh1vEJzB0OwGyXfWvP2oN4',
ACCESS_TOKEN   = '1493791152159739904-wyTZRFyyLxYqhcYu13SAgJG528a5h2',
ACCESS_TOKEN_SECRET= 'Yf29sYInnBFgFzUo7Iy2wdrWPyGhkgjlMiNpHVjUPo8R9',
BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAAI4ZQEAAAAA0CuD4%2BikWrOFi7myfncjcqXq2k8%3DjhlvJW1e8IMkuYcMeeCgqxpeoR66QgUcuUWeWRzf9Fobxmbn9L'

client = tweepy.Client(bearer_token=BEARER_TOKEN, wait_on_rate_limit=True) # need to include wait_on to prevent error popping

In [4]:
def clean(text):
    # removing @ tags and links from the text
    text= ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", text).split()) 
    
    # converting all letters to lower case and relacing '-' with spaces.
    text= text.lower().replace('-', ' ')
    
    # removing stopwards and numbers
    table= str.maketrans('', '', string.punctuation+string.digits)
    text= text.translate(table)
    
    # tokenizing words 
    tokens = word_tokenize(text)
    
    # stemming the words 
    stemmed = [porter.stem(word) for word in tokens]
    words = [w for w in stemmed if not w in n_words]

    text = ' '.join(words)
    return text

In [5]:
# Remove emojis ...
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  
        u"\U0001F300-\U0001F5FF"  
        u"\U0001F680-\U0001F6FF"  
        u"\U0001F1E0-\U0001F1FF" 
        u"\U00002500-\U00002BEF"  
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

In [23]:
from nltk.util import ngrams
def generate_clean_ngrams(tweet_text, n):
    
    # identify numbers to later remove words with numbers
    pattern = '[0-9]'
    
    # convert words to lower case
    tweet_text = tweet_text.lower()
    
    # replace special characters with a space
    tweet_text = re.sub(r'[^a-zA-Z0-9\s]', ' ', tweet_text)
    
    # replace numbers with string 'http' to later remove both links 
    # and words with numbers in one line
    tweet_text = re.sub(pattern, 'http', tweet_text)
    
    tokens = [token for token in tweet_text.split(' ') 
              if token != '' 
              and token not in stop_words 
              and 'http' not in token]
    
    output = list(ngrams(tokens, n))
    
    return output

In [134]:
def crawl_data(query, limit):
    all_tweets = []
    for tweet in tweepy.Paginator(client.search_recent_tweets,query=query,tweet_fields=['author_id','lang', 'created_at','public_metrics'], max_results=100).flatten(limit=limit):
        
        # Check if tweets is in english language and filter off RT aka retweets
        if tweet.lang == 'en' and 'RT' not in tweet.text:
            
            tweet_id = tweet.id
            user_id =  tweet.author_id
            text = tweet.text
            creation_time = tweet.created_at
            
            public_metrics = tweet.public_metrics
            retweet_count = public_metrics['retweet_count']
            like_count = public_metrics['like_count']
            
            user = client.get_user(id=user_id, user_fields=['location']).data
            username = user['username']
            location = user['location']
            all_tweets.append([tweet_id, user_id, username, text, like_count, retweet_count, location, creation_time])
            
    data = pd.DataFrame(all_tweets, columns= ['tweet_id', 'user_id','username','text','like_count', 'retweet_count', 'location','creation_date_time'])
    return data

## Crawling of Pfizer vaccine related tweets

In [46]:
query = 'Pfizer/BioNTech vaccine OR Pfizer-BioNTech vaccine OR Pfizer vaccine OR BioNTech vaccine OR pfizer lang:en'
pfizer_data = crawl_data(query, 20000)
pfizer_data

,tweet_id,user_id,text,like_count,retweet_count,creation_date_time
0,1502832204283580416,1502400542609391623,@JohnPitchy @beverleyturner @pfizer @LD_Scepti...,0,0,2022-03-13 02:21:33+00:00
1,1502832184280989697,949757090716835840,@BlueSlots @MBGov @pfizer @CBCNews So sorry to...,0,0,2022-03-13 02:21:28+00:00
2,1502832184150749185,468174673,@critcher_gordon @DrKGregorevic Good question ...,0,0,2022-03-13 02:21:28+00:00
3,1502832147819864065,230753998,@jackiescott10 Pfizer have already applied to ...,0,0,2022-03-13 02:21:19+00:00
4,1502832124327514117,1480173905298038795,@BlueSlots @MBGov @pfizer @CBCNews Prayers for...,0,0,2022-03-13 02:21:14+00:00
...,...,...,...,...,...,...
4873,1502671823913500674,1270519685285425152,@ApacKojak @NikkeiAsia When you realize the pf...,0,0,2022-03-12 15:44:15+00:00
4874,1502671822504214530,1440977697342046209,@ImmaAsole @MawonaM @demant_daniel Read the Pf...,0,0,2022-03-12 15:44:15+00:00
4875,1502671818209038338,1068702606,@ProfessorFerg8 Pfizer have clawed in $40Bn wi...,0,1,2022-03-12 15:44:14+00:00
4876,1502671804128714754,716389056188317696,@TLCStruggle @peakingsassy @reneecallier @Cath...,0,0,2022-03-12 15:44:11+00:00


In [157]:
query = 'Pfizer/BioNTech vaccine OR Pfizer-BioNTech vaccine OR Pfizer vaccine OR BioNTech vaccine OR pfizer lang:en'
new_pfizer_data = crawl_data(query, 20000)
new_pfizer_data

Rate limit exceeded. Sleeping for 293 seconds.
Rate limit exceeded. Sleeping for 806 seconds.
Rate limit exceeded. Sleeping for 799 seconds.
Rate limit exceeded. Sleeping for 799 seconds.
Rate limit exceeded. Sleeping for 801 seconds.
Rate limit exceeded. Sleeping for 807 seconds.
Rate limit exceeded. Sleeping for 807 seconds.
Rate limit exceeded. Sleeping for 803 seconds.
Rate limit exceeded. Sleeping for 804 seconds.
Rate limit exceeded. Sleeping for 805 seconds.
Rate limit exceeded. Sleeping for 804 seconds.
Rate limit exceeded. Sleeping for 825 seconds.
Rate limit exceeded. Sleeping for 826 seconds.
Rate limit exceeded. Sleeping for 827 seconds.
Rate limit exceeded. Sleeping for 825 seconds.


,tweet_id,user_id,username,text,like_count,retweet_count,location,creation_date_time
0,1502972060200574980,1241174508293312513,AngelaC11111101,The start was given! Wait... https://t.co/jIw...,0,0,None,2022-03-13 11:37:17+00:00
1,1502972032375754757,54516759,gladeolie,"Did #Pfizer #Vaccine #Documents Reveal 1,291 S...",0,0,cyberspace,2022-03-13 11:37:11+00:00
2,1502972013647994880,156451829,ctramseur,@jilevin Maybe it wasn't the same vaccine. Yes...,0,0,None,2022-03-13 11:37:06+00:00
3,1502972010892255235,1009545566290837504,SystemLordMoot,"@PianoGeek420 Got the triple Pfizer here, and ...",0,0,"England, United Kingdom",2022-03-13 11:37:05+00:00
4,1502971982673027075,1144167004892467200,DMB12445,Pfizer Buried Their Data in the Authorization ...,0,0,None,2022-03-13 11:36:59+00:00
...,...,...,...,...,...,...,...,...
4452,1502792330348216325,1431762585519595520,WaitingForPerot,@davey3483 @cheviotsue @Jerome95205658 @DagsWy...,0,0,None,2022-03-12 23:43:06+00:00
4453,1502792313021423616,3267880214,govtflu,@FOXLA Thanks Pfizer 😂,4,0,Earth,2022-03-12 23:43:02+00:00
4454,1502792277277790212,1456321142453673984,Tshepp_90,"Watch ""The Pfizer documents"" on YouTube https:...",1,0,None,2022-03-12 23:42:54+00:00
4455,1502792251721846786,1233974172696686593,Latifa2390,@opsoms @WHO @CDCgov @pfizer @FDA @EMA_news @O...,0,0,None,2022-03-12 23:42:47+00:00


In [158]:
new_pfizer_data.drop_duplicates(subset=['text'], inplace=True)
new_pfizer_data

,tweet_id,user_id,username,text,like_count,retweet_count,location,creation_date_time
0,1502972060200574980,1241174508293312513,AngelaC11111101,The start was given! Wait... https://t.co/jIw...,0,0,None,2022-03-13 11:37:17+00:00
1,1502972032375754757,54516759,gladeolie,"Did #Pfizer #Vaccine #Documents Reveal 1,291 S...",0,0,cyberspace,2022-03-13 11:37:11+00:00
2,1502972013647994880,156451829,ctramseur,@jilevin Maybe it wasn't the same vaccine. Yes...,0,0,None,2022-03-13 11:37:06+00:00
3,1502972010892255235,1009545566290837504,SystemLordMoot,"@PianoGeek420 Got the triple Pfizer here, and ...",0,0,"England, United Kingdom",2022-03-13 11:37:05+00:00
4,1502971982673027075,1144167004892467200,DMB12445,Pfizer Buried Their Data in the Authorization ...,0,0,None,2022-03-13 11:36:59+00:00
...,...,...,...,...,...,...,...,...
4452,1502792330348216325,1431762585519595520,WaitingForPerot,@davey3483 @cheviotsue @Jerome95205658 @DagsWy...,0,0,None,2022-03-12 23:43:06+00:00
4453,1502792313021423616,3267880214,govtflu,@FOXLA Thanks Pfizer 😂,4,0,Earth,2022-03-12 23:43:02+00:00
4454,1502792277277790212,1456321142453673984,Tshepp_90,"Watch ""The Pfizer documents"" on YouTube https:...",1,0,None,2022-03-12 23:42:54+00:00
4455,1502792251721846786,1233974172696686593,Latifa2390,@opsoms @WHO @CDCgov @pfizer @FDA @EMA_news @O...,0,0,None,2022-03-12 23:42:47+00:00


In [159]:
# After dropping duplicates, check to see if the text have any duplicates
new_pfizer_data.duplicated(subset=['text']).any()

False

## Saving Pfizer data to csv

In [161]:
# Convert it to CSV
new_pfizer_data.to_csv("Pfizer.csv", encoding='utf-8', index=False)

## Crawling of Sinovac vaccine related tweets

In [152]:
query = 'Sinovac-CoronaVac vaccine OR Sinovac vaccine OR CoronaVac vaccine OR sinovac lang:en'
sinovac_data = crawl_data(query, 20000)
sinovac_data

Rate limit exceeded. Sleeping for 180 seconds.
Rate limit exceeded. Sleeping for 816 seconds.
Rate limit exceeded. Sleeping for 806 seconds.


,tweet_id,user_id,username,text,like_count,retweet_count,location,creation_date_time
0,1502959853635239940,84204994,mackattackone,@GreenEntreprise @chiu_francesca @tomgrundy @h...,0,0,None,2022-03-13 10:48:47+00:00
1,1502958946377740288,697443656706453504,RockTrolla,@DrEricDing Only 50% sinovac. \nIs it Right?,0,0,Ubicuo,2022-03-13 10:45:11+00:00
2,1502954945712033792,1231486724087222272,Transit_Jam,@FY4Chan @chiu_francesca The nursing homes pro...,0,0,Hong Kong,2022-03-13 10:29:17+00:00
3,1502953689643986944,742763463597031424,AFranzsen,@AswegenElvera @Bhekisisa_MG @HealthZA @miamal...,0,0,None,2022-03-13 10:24:17+00:00
4,1502951039183556609,969429491855618048,KCMO_Neal,@DrEricDing Could this just be due to the wani...,2,0,"Kansas City, MO",2022-03-13 10:13:45+00:00
...,...,...,...,...,...,...,...,...
917,1500446344443400193,1377836392902889472,foxosk1,@Hongkong_Bear @ThomasHHChan @primroseriordan ...,1,1,None,2022-03-06 12:21:00+00:00
918,1500440744233877504,1377836392902889472,foxosk1,@Hongkong_Bear @ThomasHHChan @primroseriordan ...,1,1,None,2022-03-06 11:58:44+00:00
919,1500439703786606597,462111175,Tonko35,"Telegram\nmRNA Death Tall\n\n💉☠ In China, a 14...",0,0,"Zagreb, Croatia",2022-03-06 11:54:36+00:00
920,1500438679029833729,975566592720306177,pharmaniaga_bhd,After the CoronaVac vaccine was granted condit...,1,1,"Selangor, Malaysia",2022-03-06 11:50:32+00:00


In [153]:
# check to see if the text have any duplicates
sinovac_data.duplicated(subset=['text']).any()

True

In [154]:
sinovac_data.drop_duplicates(subset=['text'], inplace=True)
sinovac_data

,tweet_id,user_id,username,text,like_count,retweet_count,location,creation_date_time
0,1502959853635239940,84204994,mackattackone,@GreenEntreprise @chiu_francesca @tomgrundy @h...,0,0,None,2022-03-13 10:48:47+00:00
1,1502958946377740288,697443656706453504,RockTrolla,@DrEricDing Only 50% sinovac. \nIs it Right?,0,0,Ubicuo,2022-03-13 10:45:11+00:00
2,1502954945712033792,1231486724087222272,Transit_Jam,@FY4Chan @chiu_francesca The nursing homes pro...,0,0,Hong Kong,2022-03-13 10:29:17+00:00
3,1502953689643986944,742763463597031424,AFranzsen,@AswegenElvera @Bhekisisa_MG @HealthZA @miamal...,0,0,None,2022-03-13 10:24:17+00:00
4,1502951039183556609,969429491855618048,KCMO_Neal,@DrEricDing Could this just be due to the wani...,2,0,"Kansas City, MO",2022-03-13 10:13:45+00:00
...,...,...,...,...,...,...,...,...
917,1500446344443400193,1377836392902889472,foxosk1,@Hongkong_Bear @ThomasHHChan @primroseriordan ...,1,1,None,2022-03-06 12:21:00+00:00
918,1500440744233877504,1377836392902889472,foxosk1,@Hongkong_Bear @ThomasHHChan @primroseriordan ...,1,1,None,2022-03-06 11:58:44+00:00
919,1500439703786606597,462111175,Tonko35,"Telegram\nmRNA Death Tall\n\n💉☠ In China, a 14...",0,0,"Zagreb, Croatia",2022-03-06 11:54:36+00:00
920,1500438679029833729,975566592720306177,pharmaniaga_bhd,After the CoronaVac vaccine was granted condit...,1,1,"Selangor, Malaysia",2022-03-06 11:50:32+00:00


In [155]:
# check to see if the text have any duplicates
sinovac_data.duplicated(subset=['text']).any()

False

## Saving the Sinovac data to csv

In [156]:
# Convert it to CSV
sinovac_data.to_csv("Sinovac.csv", encoding='utf-8', index=False)

## Crawling of Moderna vaccine related tweets


In [147]:
query = 'Moderna vaccine OR moderna OR Moderna Covid-19 vaccine lang:en'
moderna_data = crawl_data(query, 20000)
moderna_data

Rate limit exceeded. Sleeping for 475 seconds.
Rate limit exceeded. Sleeping for 804 seconds.
Rate limit exceeded. Sleeping for 797 seconds.
Rate limit exceeded. Sleeping for 802 seconds.
Rate limit exceeded. Sleeping for 793 seconds.
Rate limit exceeded. Sleeping for 785 seconds.
Rate limit exceeded. Sleeping for 794 seconds.


,tweet_id,user_id,username,text,like_count,retweet_count,location,creation_date_time
0,1502933227991822338,954212951497564160,TimmyTate66,@PianoGeek420 Double vaxxed w Pfizer and boost...,0,0,"Tennessee, USA",2022-03-13 09:02:59+00:00
1,1502933156655435777,1320331014124695552,TheosTina,@COORY4RYAN Moderna tried for 10 years to make...,0,0,None,2022-03-13 09:02:42+00:00
2,1502933102498181122,1466683352505749507,thetruthb3t0ld,@fedupmom12 @CensoredThinker Moderna and NIH w...,0,0,None,2022-03-13 09:02:29+00:00
3,1502932998215417856,26347653,AutomaticEarth,Debt Rattle March 13 2022 #ColdWar #FiatCurren...,0,0,Atlantic Ocean,2022-03-13 09:02:04+00:00
4,1502932679066624001,933450200361947136,MoveOverThere,@SueC00K @MailOnline That may not be the bigge...,0,0,UK,2022-03-13 09:00:48+00:00
...,...,...,...,...,...,...,...,...
2195,1502419212844744707,24195285,reinoe,Pfizer delists from SEC stock exchange in comp...,1,0,"Austin, TX",2022-03-11 23:00:28+00:00
2196,1502419180129337349,1463342010463494152,StockPlungeBot,Moderna Inc. (MRNA) plunges -9.37% between 202...,0,0,None,2022-03-11 23:00:20+00:00
2197,1502419129361264640,1439398920732307456,LeighDrummond,"What actual evidence of efficacy did Pfizer, M...",0,0,None,2022-03-11 23:00:08+00:00
2198,1502419018581479428,1102671699424825344,rubic3n,And so we come full circle\nWEF digital ID. So...,2,4,Moon,2022-03-11 22:59:42+00:00


In [148]:
# check to see if the text have any duplicates
moderna_data.duplicated(subset=['text']).any()

True

In [149]:
moderna_data.drop_duplicates(subset=['text'], inplace=True)
moderna_data

,tweet_id,user_id,username,text,like_count,retweet_count,location,creation_date_time
0,1502933227991822338,954212951497564160,TimmyTate66,@PianoGeek420 Double vaxxed w Pfizer and boost...,0,0,"Tennessee, USA",2022-03-13 09:02:59+00:00
1,1502933156655435777,1320331014124695552,TheosTina,@COORY4RYAN Moderna tried for 10 years to make...,0,0,None,2022-03-13 09:02:42+00:00
2,1502933102498181122,1466683352505749507,thetruthb3t0ld,@fedupmom12 @CensoredThinker Moderna and NIH w...,0,0,None,2022-03-13 09:02:29+00:00
3,1502932998215417856,26347653,AutomaticEarth,Debt Rattle March 13 2022 #ColdWar #FiatCurren...,0,0,Atlantic Ocean,2022-03-13 09:02:04+00:00
4,1502932679066624001,933450200361947136,MoveOverThere,@SueC00K @MailOnline That may not be the bigge...,0,0,UK,2022-03-13 09:00:48+00:00
...,...,...,...,...,...,...,...,...
2195,1502419212844744707,24195285,reinoe,Pfizer delists from SEC stock exchange in comp...,1,0,"Austin, TX",2022-03-11 23:00:28+00:00
2196,1502419180129337349,1463342010463494152,StockPlungeBot,Moderna Inc. (MRNA) plunges -9.37% between 202...,0,0,None,2022-03-11 23:00:20+00:00
2197,1502419129361264640,1439398920732307456,LeighDrummond,"What actual evidence of efficacy did Pfizer, M...",0,0,None,2022-03-11 23:00:08+00:00
2198,1502419018581479428,1102671699424825344,rubic3n,And so we come full circle\nWEF digital ID. So...,2,4,Moon,2022-03-11 22:59:42+00:00


In [150]:
# check to see if the text have any duplicates
moderna_data.duplicated(subset=['text']).any()

False

## Saving the moderna data to csv

In [151]:
# Convert it to CSV
moderna_data.to_csv("Moderna.csv", encoding='utf-8', index=False)

## Crawling Covaxin vaccine related tweets

In [142]:
query = 'Covaxin vaccine OR covaxin lang:en'
new_covaxin_data = crawl_data(query, 20000)
new_covaxin_data

Rate limit exceeded. Sleeping for 664 seconds.
Rate limit exceeded. Sleeping for 810 seconds.
Rate limit exceeded. Sleeping for 805 seconds.
Rate limit exceeded. Sleeping for 804 seconds.
Rate limit exceeded. Sleeping for 813 seconds.
Rate limit exceeded. Sleeping for 808 seconds.
Rate limit exceeded. Sleeping for 813 seconds.
Rate limit exceeded. Sleeping for 805 seconds.


,tweet_id,user_id,username,text,like_count,retweet_count,location,creation_date_time
0,1502900173445296136,489184831,chovue,@CaptainXemo @WChrisNunn sigh.. youre the one...,0,0,None,2022-03-13 06:51:38+00:00
1,1502899182910119937,1389478820676210688,india_vac,BANGALORE: 44 slots found for age 18+!\nMar 13...,0,0,None,2022-03-13 06:47:42+00:00
2,1502897767315767298,1396115471703805953,CovidvaxDEL,North delhi has at least 2721 new slots availa...,0,0,"New Delhi, India",2022-03-13 06:42:04+00:00
3,1502897042334519301,489184831,chovue,@CaptainXemo @WChrisNunn jnj is based on adeno...,0,0,None,2022-03-13 06:39:12+00:00
4,1502896817314607107,3259025400,CaptainXemo,@chovue @WChrisNunn I don't care about covaxin...,0,0,None,2022-03-13 06:38:18+00:00
...,...,...,...,...,...,...,...,...
2382,1500404994268303361,1244878602094530560,Rajnees59765724,@PMOIndia when covaxin will be accepted in Ger...,0,0,None,2022-03-06 09:36:41+00:00
2383,1500403416719593472,1406895607680102404,vaxblr,18-44 #BBMP #Bengaluru #CovidVaccine Availabil...,1,0,"Bengaluru, India",2022-03-06 09:30:25+00:00
2384,1500401792856768512,1389478820676210688,india_vac,BANGALORE: 87 slots found for age 18+!\nMar 06...,0,0,None,2022-03-06 09:23:58+00:00
2385,1500396670374219779,4806490885,abyisnow,@DCEastSinghbhum Kindly arrange for Covaxin pr...,0,0,None,2022-03-06 09:03:36+00:00


In [143]:
# check to see if the text have any duplicates
new_covaxin_data.duplicated(subset=['text']).any()

True

In [144]:
new_covaxin_data.drop_duplicates(subset=['text'], inplace=True)
new_covaxin_data

,tweet_id,user_id,username,text,like_count,retweet_count,location,creation_date_time
0,1502900173445296136,489184831,chovue,@CaptainXemo @WChrisNunn sigh.. youre the one...,0,0,None,2022-03-13 06:51:38+00:00
1,1502899182910119937,1389478820676210688,india_vac,BANGALORE: 44 slots found for age 18+!\nMar 13...,0,0,None,2022-03-13 06:47:42+00:00
2,1502897767315767298,1396115471703805953,CovidvaxDEL,North delhi has at least 2721 new slots availa...,0,0,"New Delhi, India",2022-03-13 06:42:04+00:00
3,1502897042334519301,489184831,chovue,@CaptainXemo @WChrisNunn jnj is based on adeno...,0,0,None,2022-03-13 06:39:12+00:00
4,1502896817314607107,3259025400,CaptainXemo,@chovue @WChrisNunn I don't care about covaxin...,0,0,None,2022-03-13 06:38:18+00:00
...,...,...,...,...,...,...,...,...
2382,1500404994268303361,1244878602094530560,Rajnees59765724,@PMOIndia when covaxin will be accepted in Ger...,0,0,None,2022-03-06 09:36:41+00:00
2383,1500403416719593472,1406895607680102404,vaxblr,18-44 #BBMP #Bengaluru #CovidVaccine Availabil...,1,0,"Bengaluru, India",2022-03-06 09:30:25+00:00
2384,1500401792856768512,1389478820676210688,india_vac,BANGALORE: 87 slots found for age 18+!\nMar 06...,0,0,None,2022-03-06 09:23:58+00:00
2385,1500396670374219779,4806490885,abyisnow,@DCEastSinghbhum Kindly arrange for Covaxin pr...,0,0,None,2022-03-06 09:03:36+00:00


In [145]:
# check to see if the text have any duplicates
new_covaxin_data.duplicated(subset=['text']).any()

False

## Saving the Covaxin data to csv

In [146]:
# Convert it to CSV
new_covaxin_data.to_csv("Covaxin.csv", encoding='utf-8', index=False)

## Crawing Sputnik V vaccine data related tweets

In [78]:
query = 'Sputnik V vaccine OR Sputnik vaccine OR sputnik V OR russia sputnik vaccine lang:en'
sputnik_data = crawl_data(query, limit=20000)
sputnik_data

,tweet_id,user_id,text,like_count,retweet_count,creation_date_time
0,1502846122129334276,1199410182230548480,@sbprme @MootIreland Anyway sanctions have alr...,0,0,2022-03-13 03:16:51+00:00
1,1502845215161602050,1502842093592580103,@ChuckCallesto Tucker admits to being Russian ...,0,0,2022-03-13 03:13:15+00:00
2,1502842898362306562,34245009,#COVID19: #India's vaccination coverage crosse...,5,0,2022-03-13 03:04:03+00:00
3,1502839375775862790,211846145,Sputnik V: The Story Of Russia's Controversial...,0,0,2022-03-13 02:50:03+00:00
4,1502837206536257543,2342883930,#ModiRegime's love for #Sputnik vaccine in the...,0,0,2022-03-13 02:41:26+00:00
...,...,...,...,...,...,...
480,1500334105107607553,1488881366380994564,@janbobrowicz @McDutchoven @V_SCI_lNTEL @Lissy...,0,0,2022-03-06 04:55:00+00:00
481,1500333845123682304,3115671878,#Putin gets a #vaccine boost with #Sputnik Lig...,0,0,2022-03-06 04:53:58+00:00
482,1500320738997329921,291895361,@MiaFarrow @Pontifex @tribelaw @pblowe2 @RepKi...,0,0,2022-03-06 04:01:53+00:00
483,1500320265326272522,43855487,The DCGI had on February 4 granted emergency u...,5,0,2022-03-06 04:00:00+00:00


In [135]:
query = 'Sputnik V vaccine OR Sputnik vaccine OR sputnik V OR russia sputnik vaccine lang:en'
new_sputnik_data = crawl_data(query, limit=20000)
new_sputnik_data

Rate limit exceeded. Sleeping for 796 seconds.


,tweet_id,user_id,username,text,like_count,retweet_count,location,creation_date_time
0,1502885637086609411,1455052349228023814,AndrewTRei1,"I will correctly remind people, yet again, tha...",2,1,None,2022-03-13 05:53:52+00:00
1,1502873976716107776,1498608701653102594,hamzaouihazim0,Local businessman first to take Sputnik V jab ...,0,0,None,2022-03-13 05:07:32+00:00
2,1502854289064173580,269110574,PolStuff,Could it be that Putin has gone insane from ta...,0,0,None,2022-03-13 03:49:18+00:00
3,1502846122129334276,1199410182230548480,cealaigi,@sbprme @MootIreland Anyway sanctions have alr...,2,1,None,2022-03-13 03:16:51+00:00
4,1502845215161602050,1502842093592580103,chicken_mcnug_,@ChuckCallesto Tucker admits to being Russian ...,1,1,None,2022-03-13 03:13:15+00:00
...,...,...,...,...,...,...,...,...
469,1500375433665257476,961331673152487426,Anymous84861064,@ShegaonkarSunil @conspiracytalk9 @greenboat P...,0,0,In orbit around G2 V Star.,2022-03-06 07:39:13+00:00
470,1500374067903275008,60444865,KittyOwner_Club,Indian Government Recommends Permission for Ph...,0,0,Thailand,2022-03-06 07:33:48+00:00
471,1500373492952612864,1334654385863811072,MasaruTogo,Indian Government Recommends Permission for Ph...,0,0,Tokyo,2022-03-06 07:31:30+00:00
472,1500372873948499978,34262462,SputnikInt,Indian Government Recommends Permission for Ph...,38,9,None,2022-03-06 07:29:03+00:00


In [137]:
# check to see if the text have any duplicates
new_sputnik_data.duplicated(subset=['text']).any()

True

In [138]:
new_sputnik_data.drop_duplicates(subset=['text'], inplace=True)
new_sputnik_data

,tweet_id,user_id,username,text,like_count,retweet_count,location,creation_date_time
0,1502885637086609411,1455052349228023814,AndrewTRei1,"I will correctly remind people, yet again, tha...",2,1,None,2022-03-13 05:53:52+00:00
1,1502873976716107776,1498608701653102594,hamzaouihazim0,Local businessman first to take Sputnik V jab ...,0,0,None,2022-03-13 05:07:32+00:00
2,1502854289064173580,269110574,PolStuff,Could it be that Putin has gone insane from ta...,0,0,None,2022-03-13 03:49:18+00:00
3,1502846122129334276,1199410182230548480,cealaigi,@sbprme @MootIreland Anyway sanctions have alr...,2,1,None,2022-03-13 03:16:51+00:00
4,1502845215161602050,1502842093592580103,chicken_mcnug_,@ChuckCallesto Tucker admits to being Russian ...,1,1,None,2022-03-13 03:13:15+00:00
...,...,...,...,...,...,...,...,...
469,1500375433665257476,961331673152487426,Anymous84861064,@ShegaonkarSunil @conspiracytalk9 @greenboat P...,0,0,In orbit around G2 V Star.,2022-03-06 07:39:13+00:00
470,1500374067903275008,60444865,KittyOwner_Club,Indian Government Recommends Permission for Ph...,0,0,Thailand,2022-03-06 07:33:48+00:00
471,1500373492952612864,1334654385863811072,MasaruTogo,Indian Government Recommends Permission for Ph...,0,0,Tokyo,2022-03-06 07:31:30+00:00
472,1500372873948499978,34262462,SputnikInt,Indian Government Recommends Permission for Ph...,38,9,None,2022-03-06 07:29:03+00:00


In [140]:
# check to see if the text have any duplicates
new_sputnik_data.duplicated(subset=['text']).any()

False

## Saving Sputnik data to csv

In [141]:
# Convert it to CSV
new_sputnik_data.to_csv("Sputnik.csv", encoding='utf-8', index=False)

In [105]:
size_of_data = len(small_data)
print(f"The number of record for the data is {size_of_data}")

The number of record for the data is 1558


In [106]:
# Check to see if there is null values
small_data.isnull().sum()

tweet_id              0
user_id               0
text                  0
creation_date_time    0
dtype: int64

## Sentiment Analysis with textBlob packages

In [159]:
# Using TextBlob packages to generate the sentiment score for each of the text

sentiment_score = []
emot = []
for i in range(len(small_data)):
    
    small_data['text'].iloc[i] = clean(small_data['text'].iloc[i])
    small_data['text'].iloc[i] = remove_emojis(small_data['text'].iloc[i])
    analysis = TextBlob(small_data['text'].iloc[i])
    senti= analysis.sentiment.polarity
    sentiment_score.append(senti)
    
    if senti<0 :
        emotion = "NEG"
    elif senti == 0:
        emotion= "NEU"
    else:
        emotion= "POS"
        
    emot.append(emotion)

# Probably got better or cleaner ways of inserting to the columns 
small_data.insert(len(small_data.columns), "sentiment_score", sentiment_score)
small_data.insert(len(small_data.columns), "emotion", emot)

C:\Users\zappe\Anaconda3\envs\tf_gpu\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [160]:
small_data

,tweet_id,user_id,text,creation_date_time,emotion,sentiment_score
0,1495058369371070465,573428759,covid hair loss grey,2022-02-19 15:31:06+00:00,NEG,-0.050000
1,1495058368901402626,1219251620938207232,trotski ppl lost job due covid shutdown pretti...,2022-02-19 15:31:06+00:00,POS,0.108333
2,1495058368800645122,1127276173858353159,mean man washington nation game highth covid s...,2022-02-19 15:31:06+00:00,NEG,-0.356250
3,1495058367215190016,1246950247609360385,bru becaus usa polit state wa born war old glo...,2022-02-19 15:31:06+00:00,NEG,-0.008333
4,1495058363608186885,28137737,freighter travel agenc specialist cargo ship c...,2022-02-19 15:31:05+00:00,NEU,0.000000
5,1495058361712353284,1173659628179197954,covid mrna inject product caus ye caus immun s...,2022-02-19 15:31:04+00:00,NEU,0.000000
6,1495058360961482754,5851422,die covid would peopl put mask save babi,2022-02-19 15:31:04+00:00,NEU,0.000000
7,1495058358537179137,1375148797450452993,big compani drop vaccin requir worker,2022-02-19 15:31:04+00:00,NEU,0.000000
8,1495058358050725893,70394965,covid kenya record new case death recoveri,2022-02-19 15:31:04+00:00,POS,0.136364
9,1495058356330971136,1454189892532150282,warmington board plane choos dont take jab won...,2022-02-19 15:31:03+00:00,POS,0.228571


In [161]:
texts = small_data['text'].str.lower()

word_counts = Counter(word_tokenize('\n'.join(texts)))

num_unique_word = word_counts.most_common()
print(f"Total Unique Word Count: {len(num_unique_word)}")

Total Unique Word Count: 569


In [162]:
small_data['word_length'] = small_data.text.apply(lambda x: len(x.split()))

In [163]:
print(f"Total sum of words in the in the dataset is {small_data['word_length'].sum()}")

Total sum of words in the in the dataset is 902
